# train

In [ ]:
import torch

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
else:
    raise RuntimeError("Chưa bật GPU!")

!pip install --no-deps "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

!pip install --no-deps unsloth_zoo

!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes

!pip install "transformers>=4.43.3"

print("Đã cài xong thư viện!")

In [ ]:
import json
import random
from datasets import Dataset

def generate_human_centric_adaptive_data(num_samples=3000):
    data = []

    knowledge_map = {
        "Frontend": {
            "React Hooks": ["JS ES6 Functions", "Closures", "State Management Basics"],
            "CSS Grid": ["CSS Box Model", "Flexbox", "Responsive Design"],
            "NextJS Routing": ["React Components", "Client vs Server Side", "HTTP Methods"]
        },
        "Backend": {
            "JWT Authentication": ["HTTP Headers", "Encryption Basics", "RESTful API"],
            "TypeORM": ["SQL Basics", "OOP Classes", "Database Relations"],
            "Microservices": ["Docker Basics", "Monolith Architecture", "API Gateway"]
        },
        "CS Fundamentals": {
            "Dynamic Programming": ["Recursion", "Big O Notation", "Array Manipulation"],
            "Graph BFS/DFS": ["Queue/Stack Data Structure", "Adjacency Matrix", "Trees"],
            "Quick Sort": ["Recursion", "Divide and Conquer", "Time Complexity"]
        }
    }

    categories = list(knowledge_map.keys())

    role_descriptions = [
        "Kiến thức nền tảng bắt buộc",
        "Khái niệm tiên quyết quan trọng",
        "Cơ sở lý thuyết cốt lõi",
        "Kỹ năng bổ trợ cần thiết",
        "Yêu cầu đầu vào bắt buộc"
    ]

    for _ in range(num_samples):
        category = random.choice(categories)
        target_lesson = random.choice(list(knowledge_map[category].keys()))

        prereq_lesson = random.choice(knowledge_map[category][target_lesson])

        prereq_role_desc = random.choice(role_descriptions)

        target_theta = round(random.uniform(-3.0, 3.0), 2)
        prereq_theta = round(random.uniform(-3.0, 3.0), 2)

        user_input = f"""
BÁO CÁO HỌC TẬP:
- Bài hiện tại: {target_lesson}
- Điểm năng lực (Theta): {target_theta}

TÌNH TRẠNG KIẾN THỨC NỀN TẢNG:
- Bài '{prereq_lesson}' (Vai trò: {prereq_role_desc}) -> Trạng thái: {"HỔNG KIẾN THỨC" if prereq_theta < 0 else "TỐT"} (Theta {prereq_theta})

Hãy đưa ra quyết định điều hướng."""

        if prereq_theta < 0:
            action = "REVIEW"
            reasons = [
                f"Để học tốt '{target_lesson}', bạn cần nắm vững '{prereq_lesson}' trước vì đây là {prereq_role_desc.lower()}.",
                f"Hệ thống nhận thấy kiến thức về '{prereq_lesson}' của bạn còn lỗ hổng. Hãy ôn tập lại bài này để xây dựng nền tảng vững chắc.",
                f"Bài '{target_lesson}' yêu cầu kỹ năng về '{prereq_lesson}' (là {prereq_role_desc.lower()}), nhưng bạn chưa hoàn thành tốt phần này.",
                f"Đừng vội học '{target_lesson}'. Hãy quay lại củng cố '{prereq_lesson}' để tránh bị mất gốc kiến thức."
            ]
            explanation = random.choice(reasons)

        elif target_theta < -1.0:
            action = "REMEDIAL"
            reasons = [
                f"Có vẻ bài '{target_lesson}' đang hơi quá sức với bạn. Chúng ta hãy chậm lại và làm thêm bài tập bổ trợ nhé.",
                f"Kết quả bài tập cho thấy bạn đang gặp khó khăn với các concept trong '{target_lesson}'. Cần học kỹ lại phần lý thuyết.",
                f"Dù nền tảng đã tốt, nhưng bạn vẫn chưa làm chủ được '{target_lesson}'. Hãy dành thêm thời gian luyện tập bài này.",
                f"Hệ thống đề xuất lộ trình bổ trợ để giúp bạn vượt qua các trở ngại hiện tại trong bài '{target_lesson}'."
            ]
            explanation = random.choice(reasons)

        else:
            action = "NEXT"
            reasons = [
                f"Tuyệt vời! Bạn đã nắm vững '{target_lesson}' và chuẩn bị tốt nền tảng. Hãy tiếp tục sang bài mới.",
                f"Kiến thức nền tảng '{prereq_lesson}' vững chắc đã giúp bạn chinh phục '{target_lesson}' dễ dàng. Đủ điều kiện đi tiếp.",
                f"Mọi chỉ số đều tốt. Bạn đã sẵn sàng cho thử thách tiếp theo.",
                f"Hoàn thành xuất sắc '{target_lesson}'. Hệ thống đã mở khóa nội dung nâng cao cho bạn."
            ]
            explanation = random.choice(reasons)

        response_json = json.dumps({
            "Action": action,
            "Description": explanation
        }, ensure_ascii=False)

        data.append({"user": user_input, "response": response_json})

    return data

raw_data = generate_human_centric_adaptive_data(3000)
dataset = Dataset.from_list(raw_data)

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None
load_in_4bit = True

# Load Qwen 2.5 1.5B
model, tokenizer = FastLanguageModel.from_pretrained(
    "unsloth/Qwen2.5-1.5B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# Cấu hình LoRA NÂNG CAO
model = FastLanguageModel.get_peft_model(
    model,
    r = 32, 
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",], 
    lora_alpha = 64, # Alpha = r * 2
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

print("Model Configured for Reasoning!")

In [ ]:
system_prompt = """You are an AI Adaptive Learning Engine.
Analyze the student's theta scores and Knowledge Graph to determine the next step.
Output format: JSON."""

def formatting_prompts_func(examples):
    conversations = []
    for user, response in zip(examples["user"], examples["response"]):
        conversation = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user},
            {"role": "assistant", "content": response},
        ]
        conversations.append(conversation)
    return { "text" : [tokenizer.apply_chat_template(c, tokenize=False) for c in conversations] }

dataset = dataset.map(formatting_prompts_func, batched = True)

from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 8, 
        warmup_steps = 20,
        num_train_epochs = 3,           
        learning_rate = 5e-5,           

        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine", 
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

print("🚀 Bắt đầu Training")
trainer.train()

In [ ]:
FastLanguageModel.for_inference(model)

msg_content = f"""INPUT DATA:
- Lesson: Đạo hàm (Theta: -2.0)
- Prerequisite: Giới hạn (Theta: -1.5)
- Domain: Mathematics

Analyze and return JSON."""

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": msg_content}
]

# Tokenize
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

# Generate
outputs = model.generate(
    input_ids = inputs,
    max_new_tokens = 256,
    use_cache = True,
    temperature = 0.1, 
    do_sample = False  
)

# Decode
clean_response = tokenizer.decode(outputs[0][inputs.shape[1]:], skip_special_tokens=True)

print("Kết quả dự đoán:")
print(clean_response)

In [ ]:
save_path = "adaptive_lora_qwen"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"Đã lưu model tại: {save_path}")

!zip -r adaptive_lora_qwen.zip adaptive_lora_qwen

from google.colab import files
files.download("adaptive_lora_qwen.zip")


# implement

In [ ]:
# 1. Cài đặt thư viện AI 
!pip install --no-deps "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps unsloth_zoo xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install "transformers>=4.43.3"

# 2. Cài đặt thư viện Server & Tunnel
!pip install fastapi uvicorn pyngrok nest-asyncio

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
%%writefile server.py
import os
import json
import re
import torch
import uvicorn
from typing import Dict, Any, Optional, List
from contextlib import asynccontextmanager

from fastapi import FastAPI, HTTPException
from fastapi.concurrency import run_in_threadpool
from pydantic import BaseModel

from unsloth import FastLanguageModel
from peft import PeftModel
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

adaptive_instance = None

class AdaptiveModel:
    def __init__(self):
        print("AdaptiveModel: Đang khởi động Engine...")

        # CẤU HÌNH MODEL
        base_model_id = "unsloth/Qwen2.5-1.5B-Instruct"
        adapter_path = "/content/drive/MyDrive/edtech/adaptive_lora_qwen"
        if not os.path.exists(adapter_path):
             adapter_path = "adaptive_lora_qwen"

        max_seq_length = 2048
        dtype = None
        load_in_4bit = True

        try:
            self.model, self.tokenizer = FastLanguageModel.from_pretrained(
                model_name=base_model_id,
                max_seq_length=max_seq_length,
                dtype=dtype,
                load_in_4bit=load_in_4bit,
            )

            if os.path.exists(adapter_path):
                print(f"Loading LoRA adapter: {adapter_path}")
                self.model = PeftModel.from_pretrained(
                    self.model,
                    adapter_path,
                    is_trainable=False,
                )
            else:
                print(f"CẢNH BÁO: Không tìm thấy '{adapter_path}'. Đang chạy Model gốc (Logic có thể kém hơn).")

            FastLanguageModel.for_inference(self.model)
            print("AdaptiveModel: Sẵn sàng!")

        except Exception as e:
            print(f"Critical Load Error: {e}")
            self.model = None

    def _build_messages(self, data: Dict[str, Any]) -> list:
        context = data.get("current_context", {})
        kg = data.get("knowledge_graph_subgraph", [])
        profile = data.get("user_mastery_profile", [])

        curr_id = str(context.get('target_content_id'))
        curr_theta = float(context.get('current_theta', 0))

        profile_map = {str(item['content_id']): item for item in profile}
        curr_info = profile_map.get(curr_id, {})
        curr_title = curr_info.get('title', f"Lesson {curr_id}")

        curr_desc = "Không có mô tả nội dung."
        for e in kg:
            if str(e.get('target_id')) == curr_id:
                curr_desc = e.get('description_target', '')
                if curr_desc: break
            elif str(e.get('source_id')) == curr_id:
                curr_desc = e.get('description_source', '')
                if curr_desc: break

        prereq_analysis = []
        for e in kg:
            if str(e['target_id']) == curr_id and e['type'] == 'PREREQUISITE':
                src_id = str(e['source_id'])
                src_title = e.get('source', 'Unknown Lesson')

                src_content = e.get('description_source', 'Không có mô tả chi tiết.')

                p_theta = profile_map.get(src_id, {}).get('theta', 0.0)
                status = "HỔNG KIẾN THỨC" if p_theta < 0 else "NẮM VỮNG"

                prereq_analysis.append(
                    f"- Bài: {src_title} (ID: {src_id})\n"
                    f"  + Nội dung kiến thức: {src_content}\n"
                    f"  + Trạng thái học viên: Theta {p_theta} ({status})"
                )

        next_analysis = []
        for e in kg:
            if str(e['source_id']) == curr_id:
                tgt_id = str(e['target_id'])
                tgt_title = e.get('target', 'Unknown Next Lesson')
                tgt_content = e.get('description_target', 'Nội dung bài tiếp theo')
                next_analysis.append(f"- {tgt_title} (ID: {tgt_id}): {tgt_content}")

        system_prompt = """Bạn là  một giảng viên, hãy đưa ra chiến lược giúp học viên hiểu sâu và bền vững hơn về bài học
CHIẾN LƯỢC TƯ DUY:
- Đọc nội dung bài học hiện tại và các bài liên quan để hiểu kiến thức cốt lõi đang được sử dụng.
- So sánh nội dung kiến thức nền tảng với mức độ hiểu hiện tại của học viên.
  Nếu học viên chưa nắm được các khái niệm cần thiết cho bài hiện tại, việc học tiếp sẽ kém hiệu quả → ưu tiên ôn tập.
- Nếu kiến thức nền đã đủ nhưng học viên vẫn gặp khó khăn ở bài hiện tại,
  hãy xem đây là vấn đề về độ khó hoặc khả năng áp dụng → cần củng cố thêm trước khi học mới.
- Chỉ đề xuất học tiếp khi học viên thể hiện sự sẵn sàng về mặt hiểu biết,
  dựa trên mối liên hệ nội dung giữa các bài học, không chỉ dựa vào con số.

OUTPUT FORMAT (JSON Only):
{"Action": "REVIEW/REMEDIAL/NEXT", "Description": "Giải thích dựa trên mối liên hệ nội dung bài học..."}"""

        # 5. USER PROMPT
        user_prompt = f"""
--- BỐI CẢNH BÀI ĐANG HỌC ---
Tên bài: {curr_title} (ID: {curr_id})
Mô tả nội dung: {curr_desc}
Kết quả hiện tại: Theta {curr_theta}

--- PHÂN TÍCH CÁC BÀI TIỀN ĐỀ (PREREQUISITES) ---
{chr(10).join(prereq_analysis) if prereq_analysis else "Không có bài tiền đề (Đây là bài nhập môn)."}

--- CÁC LỰA CHỌN TIẾP THEO ---
{chr(10).join(next_analysis) if next_analysis else "Không có dữ liệu bài tiếp theo."}

Dựa trên nội dung và kết quả trên, hãy đưa ra quyết định."""

        return [{"role": "system", "content": system_prompt}, {"role": "user", "content": user_prompt}]
    def _resolve_target_id(self, action: str, data: Dict[str, Any]) -> str:
        kg = data.get("knowledge_graph_subgraph", [])
        profile = data.get("user_mastery_profile", [])
        curr_id = str(data["current_context"]['target_content_id'])

        # Map Theta
        p_map = {str(item['content_id']): float(item['theta']) for item in profile}

        # Case 1: REVIEW
        if action == "REVIEW":
            candidates = []
            for e in kg:
                if str(e['target_id']) == curr_id and e['type'] == 'PREREQUISITE':
                    src_id = str(e['source_id'])
                    theta = p_map.get(src_id, 0.0)
                    if theta < 0:
                        candidates.append((src_id, theta))

            if candidates:
                candidates.sort(key=lambda x: x[1])
                return candidates[0][0]

            return curr_id

        # Case 2: NEXT
        if action == "NEXT":
            for e in kg:
                if str(e['source_id']) == curr_id and e['type'] == "RELATED":
                    return str(e['target_id'])

        # Case 3: REMEDIAL
        return curr_id

    def predict(self, request_data: Dict[str, Any]) -> Dict[str, Any]:
        if self.model is None:
            return self._fallback_response(request_data, "Model not loaded")

        request_id = request_data.get("request_id")

        try:
            messages = self._build_messages(request_data)

            device = self.model.device
            inputs = self.tokenizer.apply_chat_template(
                messages,
                tokenize=True,
                add_generation_prompt=True,
                return_tensors="pt"
            ).to(device)

            with torch.no_grad():
                outputs = self.model.generate(
                    input_ids=inputs,
                    max_new_tokens=256,
                    do_sample=False,
                    temperature=0.0
                )

            response_text = self.tokenizer.decode(outputs[0][inputs.shape[1]:], skip_special_tokens=True)

            # Extract JSON
            match = re.search(r'\{[\s\S]*\}', response_text)
            if match:
                ai_result = json.loads(match.group(0))
                action = ai_result.get("Action", "NEXT").upper()
                description = ai_result.get("Description", "AI Recommendation")

                final_target_id = self._resolve_target_id(action, request_data)

                if action == "REVIEW" and final_target_id == str(request_data["current_context"]['target_content_id']):
                    action = "REMEDIAL"
                    description = "Đề xuất ôn tập bài hiện tại."

                return {
                    "request_id": request_id,
                    "suggestion": {
                        "Action": action,
                        "TargetID": final_target_id,
                        "Description": description
                    },
                    "status": "SUCCESS"
                }
            else:
                raise Exception("Invalid JSON Format")

        except Exception as e:
            print(f"Fallback triggered: {e}")
            return self._fallback_response(request_data, str(e))

    def _fallback_response(self, data, error_msg):
        fallback = self._fallback_logic(data)
        return {
            "request_id": data.get("request_id"),
            "suggestion": fallback,
            "status": "SUCCESS",
            "debug_info": error_msg
        }

    def _fallback_logic(self, data: Dict[str, Any]) -> Dict[str, Any]:

      ctx = data.get("current_context", {})
      kg = data.get("knowledge_graph_subgraph", [])
      profile = data.get("user_mastery_profile", [])

      curr_id = str(ctx.get('target_content_id'))
      curr_theta = float(ctx.get('current_theta', 0))

      p_map = {str(i['content_id']): float(i['theta']) for i in profile}

      prereq_candidates = []
      for e in kg:
          if str(e.get('target_id')) == curr_id and e.get('type') == 'PREREQUISITE':
              prereq_id = str(e.get('source_id'))
              theta = p_map.get(prereq_id, 0.0)
              if theta < 0:
                  prereq_candidates.append((prereq_id, theta, e))

      if prereq_candidates:
          prereq_candidates.sort(key=lambda x: x[1])
          prereq_id, theta, edge = prereq_candidates[0]

          return {
              "Action": "REVIEW",
              "TargetID": prereq_id,
              "Description": f"Phát hiện hổng kiến thức nền tảng '{edge.get('source', '')}'"
          }

      if curr_theta < -1.0:
          for e in kg:
              if str(e.get('source_id')) == curr_id and e.get('type') == 'REMEDIAL':
                  return {
                      "Action": "REMEDIAL",
                      "TargetID": str(e.get('target_id')),
                      "Description": "Đề xuất ôn tập bài remedial do năng lực hiện tại thấp"
                  }

          return {
              "Action": "REMEDIAL",
              "TargetID": curr_id,
              "Description": "Đề xuất ôn tập lại bài hiện tại do năng lực còn yếu"
          }

      for e in kg:
          if str(e.get('source_id')) == curr_id and e.get('type') == 'RELATED':
              return {
                  "Action": "NEXT",
                  "TargetID": str(e.get('target_id')),
                  "Description": "Đủ điều kiện chuyển sang bài học tiếp theo"
              }

      return {
          "Action": "NEXT",
          "TargetID": curr_id,
          "Description": "Không tìm thấy lựa chọn phù hợp hơn, giữ nguyên bài hiện tại"
      }

@asynccontextmanager
async def lifespan(app: FastAPI):
    global adaptive_instance
    adaptive_instance = AdaptiveModel()
    yield
    print("Shutting down...")

app = FastAPI(lifespan=lifespan)

class CurrentContext(BaseModel):
    target_content_id: str
    current_theta: float

class KnowledgeGraphEdge(BaseModel):
    source_id: str
    source: str
    description_source: Optional[str] = ""
    target_id: str
    target: str
    type: str
    description_target: Optional[str] = ""

class UserMastery(BaseModel):
    content_id: str
    title: Optional[str] = None
    theta: float
    status: str

class AdaptiveRequest(BaseModel):
    request_id: str
    current_context: CurrentContext
    knowledge_graph_subgraph: List[KnowledgeGraphEdge]
    user_mastery_profile: List[UserMastery]

@app.post("/adaptive")
async def navigate_learning(request: AdaptiveRequest):
    input_data = request.model_dump()

    result = await run_in_threadpool(adaptive_instance.predict, input_data)
    return result

@app.get("/")
def home():
    return {"message": "AI Server is Online"}

In [ ]:
import nest_asyncio
from pyngrok import ngrok
import time
NGROK_TOKEN = "373glv84UEC0goVEx7XtaTAYeDd_4LkpbJTdct429XbP8QmKG"
ngrok.set_auth_token(NGROK_TOKEN)

ngrok.kill()

public_url = ngrok.connect(8000).public_url
print(f"🚀 SERVER ĐANG CHẠY TẠI PUBLIC URL: {public_url}")
print(f"👉 API Endpoint cho NestJS: {public_url}/adaptive")
print("⏳ Đang khởi động Model (Mất khoảng 1-2 phút)...")
!uvicorn server:app --host 0.0.0.0 --port 8000